In [1]:
import pandas as pd
import os
import numpy as np
import scipy as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
import pycuda.driver as cuda

In [2]:
from livelossplot import PlotLosses
from time import sleep
import timeit

In [3]:
import pyro
import pyro.distributions as dist
from pyro.distributions import Normal, Uniform, Delta
from pyro.infer.mcmc import MCMC, HMC, NUTS
from pyro.infer.mcmc.api import MCMC
import pyro.poutine as poutine
from pyro.infer import EmpiricalMarginal, SVI, Trace_ELBO, TracePredictive, JitTrace_ELBO, TraceGraph_ELBO, TraceEnum_ELBO
from pyro.optim import Adam
from pyro.infer.mcmc.util import predictive
from pyro.distributions.util import sum_rightmost
from pyro.contrib.autoguide import AutoDiagonalNormal

In [4]:
from scipy.stats import uniform, norm
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

In [5]:
# for CI testing
pyro.enable_validation(True)
pyro.set_rng_seed(1)
pyro.enable_validation(True)

In [6]:
lst_device_name = ['cuda:'+str(i) for i in range(torch.cuda.device_count())]

In [7]:
lst_device = [torch.device(name) for name in lst_device_name]

In [8]:
lst_device

[device(type='cuda', index=0), device(type='cuda', index=1)]

In [9]:
sigma_noise = 0.1
N = 200
np.random.seed(40)

In [10]:
X1 = Uniform(torch.tensor([-1.0]), torch.tensor([-0.63])).sample(sample_shape=torch.Size([int(N/2)])).squeeze()
X2 = Uniform(torch.tensor([0.45]), torch.tensor([1.1])).sample(sample_shape=torch.Size([int(N/2)])).squeeze()
X = torch.cat((X1, X2), 0)
X.detach()
pass

In [11]:
e = torch.from_numpy(norm.rvs(loc=0, scale=sigma_noise, size=N)).detach().float()
y = torch.cos(4.0*(X+0.2)) + e
y.detach()
pass

In [12]:
x_data, y_data = X.unsqueeze(1), y

In [13]:
H = 50

In [14]:
class RegressionModel(nn.Module):
    def __init__(self, device):
        super(RegressionModel, self).__init__()
        self.linear1 = nn.Linear(1, H)
        self.linear2 = nn.Linear(H,1)
        self.to(device)

    def forward(self, x):
        out = torch.tanh(self.linear1(x))
        out = self.linear2(out)
        return out

In [15]:
def model(x_data, y_data):
    options = dict(dtype=x_data.dtype, device=x_data.device)
    
    weight_loc = torch.zeros(H, 1, **options)
    weight_scale = 5*torch.ones(H, 1, **options)
    bias_loc = torch.zeros(H, **options)
    bias_scale = 5*torch.ones(H, **options)
    
    linear1_w_prior = Normal(weight_loc, weight_scale).independent(2)
    linear1_b_prior = Normal(bias_loc, bias_scale).independent(1)
    
    weight_loc = torch.zeros(1, H, **options)
    weight_scale = 5*torch.ones(1, H, **options)
    bias_loc = torch.zeros(1, **options)
    bias_scale = 5*torch.ones(1, **options)
    
    linear2_w_prior = Normal(weight_loc, weight_scale).independent(2)
    linear2_b_prior = Normal(bias_loc, bias_scale).independent(1)
    
    priors = {'linear1.weight': linear1_w_prior, 'linear1.bias': linear1_b_prior,
              'linear2.weight': linear2_w_prior, 'linear2.bias': linear2_b_prior,}
    
    lifted_module = pyro.random_module("module", regression_model, priors)
    
    lifted_reg_model = lifted_module()
    
    with pyro.plate("map", device=x_data.device):
        prediction_mean = lifted_reg_model(x_data).squeeze(-1)
        pyro.sample("obs", Normal(prediction_mean, sigma_noise), obs=y_data)
    return prediction_mean

In [16]:
def train(x, y, device):
    
    x_data = x.detach().clone().to(device)
    y_data = y.detach().clone().to(device)
    
    regression_model = RegressionModel(device)
    guide = AutoDiagonalNormal(model)
    scheduler = pyro.optim.ReduceLROnPlateau({'optimizer': torch.optim.Adam, 'optim_args': {'lr': 0.01}, 'factor': 0.5, 'patience': 2})
    svi = SVI(model, guide, scheduler, loss=Trace_ELBO(vectorize_particles=True))
    
    num_epoch = 10
    num_iterations = 1000
    for j in range(num_epoch):
        losses = [None] * num_iterations
        for k in range(num_iterations):
            loss = svi.step(x_data, y_data)
            losses[k] = loss
        scheduler.step(np.mean(losses))
        
    x_test = torch.linspace(-2.0, 2.0).unsqueeze(1).to(device)
    guide_trace = poutine.trace(guide).get_trace(x_test, None)
    return guide_trace

In [17]:
device = lst_device[1]

In [18]:
x = x_data.detach().clone().to(device)
y = y_data.detach().clone().to(device)

In [19]:
regression_model = RegressionModel(device)
guide = AutoDiagonalNormal(model)

In [ ]:
scheduler = pyro.optim.ReduceLROnPlateau({'optimizer': torch.optim.Adam, 'optim_args': {'lr': 0.01}, 'factor': 0.5, 'patience': 2})
svi = SVI(model, guide, scheduler, loss=Trace_ELBO(vectorize_particles=True))

In [ ]:
num_epoch = 10
num_iterations = 1000
for j in range(num_epoch):
    losses = [None] * num_iterations
    for k in range(num_iterations):
        loss = svi.step(x, y)
        losses[k] = loss
    scheduler.step(np.mean(losses))

In [23]:
pyro.param

<function pyro.primitives.param(name, *args, **kwargs)>

In [ ]:
guide_trace = train(x_data, y_data, lst_device[1])

In [ ]:
guide_trace

In [ ]:
posterior = svi.run(x_data, y_data)

In [ ]:
guide_trace.nodes.keys()

In [ ]:
guide_trace.nodes['auto_loc']

In [ ]:
guide_trace.nodes['module$$$linear2.weight']

In [ ]:
x_test = torch.linspace(-2.0, 2.0).unsqueeze(1).cuda()

In [ ]:
guide_trace = poutine.trace(guide).get_trace(x_test, None)

In [ ]:
lifted_reg_model = poutine.replay(model, guide_trace)

In [ ]:
y_test = lifted_reg_model(x_test, None)

In [ ]:
for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))